In [ ]:
#mount drive
from google.colab import drive
drive.mount('/content/drive',force_remount=True)

Mounted at /content/drive


In [ ]:
!pip install datasets sentencepiece --quiet
!pip install transformers[torch] --quiet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 471.6/471.6 kB 15.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 12.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 12.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 18.3 MB/s eta 0:00:00


In [ ]:
import transformers
import numpy as np
import json
import os
from torch.utils.data import Dataset, random_split
import torch
import pandas as pd
import ipywidgets as widgets
from IPython.display import display
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

from tqdm.notebook import tqdm

In [ ]:
if torch.cuda.is_available():
    print("CUDA is available. Using GPU.")
    device = torch.device("cuda")
else:
    print("CUDA not available. Using CPU.")
    device = torch.device("cpu")

CUDA is available. Using GPU.


In [ ]:
project_root = '/content/drive/MyDrive/Financial_news/'
# model_name = 'test_model_attention_mask' # @param ["test_model_attention_mask"]
model_name = 'test_model_attention_mask' # @param ["test_model_attention_mask"]
tokenizer_name = 'google/mt5-base'
model_path = os.path.join(project_root, 'model',model_name, 'save_model')
model_path

'/content/drive/MyDrive/Financial_news/model/test_model_attention_mask/save_model'

In [ ]:
# set permission
os.chmod(model_path, 755)
os.chdir(model_path)
os.getcwd()

'/content/drive/MyDrive/Financial_news/model/test_model_attention_mask/save_model'

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(tokenizer_name, use_fast=False)
SPECIAL_TOKEN = ['<SYMBOL>','<ASPECT>','<OPINION>','<POS>','<NEG>','<NEU>']
special_tokens_dict = {'additional_special_tokens': SPECIAL_TOKEN}
tokenizer.add_special_tokens(special_tokens_dict)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/376 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/702 [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/4.31M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/65.0 [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


6

In [ ]:
model = AutoModelForSeq2SeqLM.from_pretrained(model_path)

In [ ]:
# load testset

data_folder = os.path.join(project_root,'text-to-label/dataset')
file_dropdown = widgets.Dropdown(
    options=[file_name for file_name in os.listdir(data_folder) if file_name.endswith('.csv')],
    description='select dataset',
)
display(file_dropdown)

Dropdown(description='select dataset', options=('TTFJ_fillnote_label.csv', 'mdtesting - Data.csv', 'mdtesting …

In [ ]:
print(f'selected file: {file_dropdown.value}')
file_path = os.path.join(data_folder, file_dropdown.value)
df = pd.read_csv(file_path)

# df = df[['news','symbol','aspect','opinion','sentiment']]
df

selected file: md_fillnote.csv


,text,label
0,BAFS รายได้การบินพุ่ง ดันงวด 9 เดือนพลิกกำไร 6...,<SYMBOL> BAFS <ASPECT> รายได้ <OPINION> พุ่ง <...
1,BAFS รายได้การบินพุ่ง ดันงวด 9 เดือนพลิกกำไร 6...,<SYMBOL> BAFS <ASPECT> งวด <OPINION> ดัน <POS>
2,BAFS รายได้การบินพุ่ง ดันงวด 9 เดือนพลิกกำไร 6...,<SYMBOL> BAFS <ASPECT> พลิก <OPINION> กำไร <POS>
3,NOBLE เล็งเปิดตัว 2 โครงการท้ายปี มูลค่า 3.10 ...,<SYMBOL> NOBLE <ASPECT> กำไร <OPINION> ทะลัก <...
4,สรุปกระดานรายใหญ่เช้านี้ BGRIM มูลค่าสูงสุด 9 ...,<SYMBOL> BGRIM <ASPECT> มูลค่า <OPINION> สูงสุ...
...,...,...
747,MGC มั่นใจ Q4 พีกสุด! จ่อส่งมอบรถ 800 คัน ดันร...,<SYMBOL> MGC <ASPECT> Q4 <OPINION> มั่นใจ <POS>
748,MGC มั่นใจ Q4 พีกสุด! จ่อส่งมอบรถ 800 คัน ดันร...,<SYMBOL> MGC <ASPECT> รายได้ <OPINION> โตเข้าเ...
749,SCB โชว์กำไร 9 เดือนแรกโต 7% ทะลุ 3.3 หมื่นล้า...,<SYMBOL> SCB <ASPECT> กำไร <OPINION> โต <POS>
750,SCB โชว์กำไร 9 เดือนแรกโต 7% ทะลุ 3.3 หมื่นล้า...,<SYMBOL> SCB <ASPECT> กำไร <OPINION> ทะลุ <POS>


In [ ]:
def generate_text(input_text):
    input_ids = tokenizer(input_text, return_tensors="pt").input_ids

    input_ids = input_ids.to(device)
    model.to(device)
    # outputs = model.generate(input_ids, num_return_sequences=3, do_sample=True, max_length=30, temperature=1.3)
    # outputs = model.generate(input_ids, num_return_sequences=3, do_sample=True, max_length=30, temperature=1.5)
    outputs = model.generate(
    input_ids,
    num_return_sequences=5,  # เปลี่ยนเป็น 5 ผลลัพธ์
    do_sample=True,
    num_beams=5,
    max_length=50,
    temperature=0.7,  # ปรับความสร้างสรรค์ของการสุ่ม
    top_k=50,    # ควบคุมจำนวน top ตัวเลือก
    top_p=0.9         # ควบคุมโอกาสสะสม
    )
    # outputs = model.generate(input_ids, num_return_sequences=3, do_sample=True, max_length=30, top_p=0.9)
    # outputs = model.generate(input_ids, num_return_sequences=3, do_sample=True, max_length=30, top_k=50)

    # outputs = model.generate(input_ids, num_return_sequences=1, do_sample=True, max_length=30)

    output_texts = [tokenizer.decode(output).replace('<pad>','') for output in outputs]
    return tuple(output_texts)


In [ ]:
# Generate text and split into 3 separate columns
tqdm.pandas(desc='Generating texts...')
df[['generate-sample-1', 'generate-sample-2', 'generate-sample-3', 'generate-sample-4', 'generate-sample-5']] = pd.DataFrame(
    df['text'].progress_apply(lambda x: generate_text(x)).tolist(), index=df.index
)

# แสดงผลลัพธ์
print(df.head())

Generating texts...:   0%|          | 0/752 [00:00<?, ?it/s]

                                                text  \
0  BAFS รายได้การบินพุ่ง ดันงวด 9 เดือนพลิกกำไร 6...   
1  BAFS รายได้การบินพุ่ง ดันงวด 9 เดือนพลิกกำไร 6...   
2  BAFS รายได้การบินพุ่ง ดันงวด 9 เดือนพลิกกำไร 6...   
3  NOBLE เล็งเปิดตัว 2 โครงการท้ายปี มูลค่า 3.10 ...   
4  สรุปกระดานรายใหญ่เช้านี้ BGRIM มูลค่าสูงสุด 9 ...   

                                               label  \
0  <SYMBOL> BAFS <ASPECT> รายได้ <OPINION> พุ่ง <...   
1     <SYMBOL> BAFS <ASPECT> งวด <OPINION> ดัน <POS>   
2   <SYMBOL> BAFS <ASPECT> พลิก <OPINION> กำไร <POS>   
3  <SYMBOL> NOBLE <ASPECT> กำไร <OPINION> ทะลัก <...   
4  <SYMBOL> BGRIM <ASPECT> มูลค่า <OPINION> สูงสุ...   

                                   generate-sample-1  \
0   <SYMBOL> BAFS <ASPECT> รายได้ <OPINION> พุ่ง ...   
1   <SYMBOL> BAFS <ASPECT> รายได้ <OPINION> พุ่ง ...   
2   <SYMBOL> BAFS <ASPECT> รายได้ <OPINION> พุ่ง ...   
3   <SYMBOL> NOBLE <ASPECT> กําไร <OPINION> ทะลัก...   
4   <SYMBOL> BGRIM <ASPECT> มูลค่า <OPINION> ส

In [ ]:
# # ประยุกต์ generate ข้อความ
# tqdm.pandas(desc='Generating texts...')
# df[['generate-sample-1','generate-sample-2','generate-sample-3']]= df['text'].progress_apply(lambda x: generate_text(x))

In [ ]:
# import pandas as pd

# # สมมุติว่ามี DataFrame ที่ชื่อ df อยู่แล้ว

# # ขยายผลลัพธ์ที่เป็น tuple ออกมาเป็นหลายแถว
# df_exploded = df.explode('generate-result').reset_index(drop=True)

# # แสดงผลลัพธ์ที่แยกแถว
# print(df_exploded.head())


# # เปลี่ยนพาธของ data_folder
# data_folder = '/content/drive/MyDrive/Financial_news/eval'

# # สร้างพาธใหม่สำหรับไฟล์ผลลัพธ์
# output_file_path = os.path.join(data_folder, 'md_jw.csv')

# # บันทึกไฟล์ CSV
# df.to_csv(output_file_path, index=False)


# # # บันทึกผลลัพธ์ที่ขยายแถวลงในไฟล์ CSV ใหม่
# # df_exploded.to_csv('path_to_save_new_file.csv', index=False)

# # แสดงผลลัพธ์ที่แยกแถว
# print(df_exploded)

In [ ]:
# def generate_text(input_text):
#     input_ids = tokenizer(input_text, return_tensors="pt").input_ids
#     input_ids = input_ids.to(device)
#     model.to(device)

#     outputs = model.generate(input_ids, num_return_sequences=3, do_sample=True, max_length=30)
#     output_texts = [tokenizer.decode(output).replace('<pad>','').strip() for output in outputs]

#     # ตรวจสอบว่ามี 3 ผลลัพธ์หรือไม่ ถ้ามีน้อยกว่าหรือมากกว่า 3 จะเติมให้ครบหรือทำการตัด
#     while len(output_texts) < 3:
#         output_texts.append('')  # เติมช่องว่างให้ครบ 3 คำตอบ
#     return tuple(output_texts[:3])  # ตรวจสอบว่าผลลัพธ์จะถูกตัดเหลือ 3 ค่าเสมอ


In [ ]:
# input_text = ["โบรกเชียร์ “ซื้อ” SUN เป้าใหม่ 8 บาท อัพกำไรปี 66 ขึ้น 30%", "AP โบรกแนะ “ทยอยสะสม” รอรับปันผล ชูยีลด์สูง 6-7% พร้อมจับตากำไรปี 66-67 ออลไทม์ไฮ", "NSL โบรกแนะซื้อเป้า 26 บ. จับตา Q2 กำไร “นิวไฮ” แตะ 82 ล้าน ลุ้นดันทั้งปีนี้ทะลุ 343 ล้าน"]
# input_ids = tokenizer(input_text, padding=True, return_tensors="pt").input_ids
# input_ids = input_ids.to(device)
# outputs = model.generate(input_ids, num_beams=5, num_return_sequences=1, do_sample=False, max_length=50)

In [ ]:
# shape = (batch_size * num_return_sequences, max_length)
# reshape to (num_return_sequences, batch_size, max_length)
# outputs.shape

In [ ]:
# [tokenizer.decode(output).replace('<pad>','') for output in outputs]

In [ ]:
# tqdm.pandas(desc='generate texts..')

# df[['generate-sample-1','generate-sample-2','generate-sample-3']] = df['text'].progress_apply(lambda x: generate_text(x))

In [ ]:
# tqdm.pandas(desc='generate texts..')

# df[['generate-sample-1', 'generate-sample-2', 'generate-sample-3']] = df['text'].progress_apply(lambda x: generate_text(x)).apply(pd.Series)


In [ ]:
# tqdm.pandas(desc='generate texts..')

# df['generate-sample'] = df['text'].progress_apply(lambda x: generate_text(x)).apply(pd.Series)

In [ ]:
# tqdm.pandas(desc='generate texts..')
# result = pd.DataFrame()
# df['generate-result']=  df['text'].progress_apply(lambda x: generate_text(x))

In [ ]:
# Example
input_text = "โบรกเชียร์ “ซื้อ” SUN เป้าใหม่ 8 บาท อัพกำไรปี 66 ขึ้น 30%"
for text in generate_text(input_text):
  print(text)


In [ ]:
# Example
input_text = "EA ปิดร่วง 7% วิตก “กระแสเงินสด” หดตัวจากสต๊อก ฟากผู้บริหารมั่นใจงวด Q4 บุ๊ครายได้สูงขึ้น"
for text in generate_text(input_text):
  print(text)


In [ ]:
# เปลี่ยนพาธของ data_folder
data_folder = '/content/drive/MyDrive/Financial_news/eval'

# สร้างพาธใหม่สำหรับไฟล์ผลลัพธ์
output_file_path = os.path.join(data_folder, 'md_fillnote_testing5.csv')

# บันทึกไฟล์ CSV
df.to_csv(output_file_path, index=False)

# แสดงผลลัพธ์ที่บันทึก
print(f'Results saved to {output_file_path}')


Results saved to /content/drive/MyDrive/Financial_news/eval/md_fillnote_testing5.csv


In [ ]:
# output_file_path = os.path.join(data_folder, 'generated_texts.csv')
# df.to_csv(output_file_path, index=False)
# print(f'Results saved to {output_file_path}')
